In [2]:
pip install lightgbm

     ---------------------------------------- 1.0/1.0 MB 16.3 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import matplotlib.pyplot as plt

import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

from tensorflow import keras
from keras.models import Sequential
from keras import Input
from keras.layers import Dense
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.models import load_model
from keras.callbacks import EarlyStopping

from lightgbm import LGBMRegressor

In [2]:
# epochs = 100
# batch_size = 32
# drop_rate = 0.3

max_depth = 5
n_estimators = 5000             # 400
lr = 0.01
num_leaves = 31
early_stopping_rounds = 300      # 100
verbose = 500                  # True

In [3]:
curr_dir = os.getcwd()
curr_dir

'C:\\weather\\v_lgbm'

In [4]:
# path_train = os.path.join(curr_dir+"\\data_rota\\"+"\\train")
# path_test = os.path.join(curr_dir+"\\data_rota\\"+"\\test")
    
# folders_tr = os.listdir(path_train)
# folders_test = os.listdir(path_test)
    
# train = pd.DataFrame()
# for files in folders_tr:
#     df= pd.read_csv(os.path.join(curr_dir+"\\data_rota\\"+"\\train\\")+files)
#     train = pd.concat([train, df])
# train

,YearMonthDayHourMinute,STN,Lon,Lat,isitu-LST,SolarZA,SateZA,ESR,Height,LandType,...,insitu-TED5.0_new,insitu-PA_new,insitu-PS_new,mean_6_23_new,mean_34_37_new,mean_38_39_new,mean_40_42_new,mean_43_44_new,NDVI_new,NDWI_new
0,202001010000,90,128.56473,38.25085,-2.6,162.63634,44.52481,-9.38141,17.53,2,...,-999.0,1025.2,1027.5,0.583554,-999.000000,-999.000000,-999.0,0.990591,-0.333333,-999.000000
1,202001010010,90,128.56473,38.25085,-2.5,163.55127,44.52481,-9.38141,17.53,2,...,-999.0,1025.2,1027.5,0.583613,-999.000000,-999.000000,-999.0,0.990591,1.000000,-1.000000
2,202001010020,90,128.56473,38.25085,-2.7,164.22845,44.52481,-9.38141,17.53,2,...,-999.0,1024.9,1027.2,0.583406,-999.000000,-999.000000,-999.0,0.990301,-7.206897,-1.000000
3,202001010030,90,128.56473,38.25085,-2.7,164.63608,44.52481,-9.38141,17.53,2,...,-999.0,1024.9,1027.2,0.583700,-999.000000,-999.000000,-999.0,0.990301,-7.000000,-0.273743
4,202001010040,90,128.56473,38.25085,-2.7,164.57039,44.52481,-9.38141,17.53,2,...,-999.0,1025.0,1027.3,0.584239,-999.000000,-999.000000,-999.0,0.990398,-0.881423,-0.273743
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296417,202108202310,980,126.48951,33.26062,27.3,128.95641,38.90221,-5.79447,141.84,2,...,-999.0,-999.0,-999.0,0.523238,0.681095,0.778348,-999.0,-999.000000,0.000000,1.000000
296418,202108202320,980,126.48951,33.26062,27.3,129.99785,38.90221,-5.79447,141.84,2,...,-999.0,-999.0,-999.0,0.546689,0.680646,0.778348,-999.0,-999.000000,-0.600000,0.532787
296419,202108202330,980,126.48951,33.26062,27.3,130.94579,38.90221,-5.79447,141.84,2,...,-999.0,-999.0,-999.0,0.537296,0.680646,0.778348,-999.0,-999.000000,1.000000,0.532787
296420,202108202340,980,126.48951,33.26062,27.2,131.79460,38.90221,-5.79447,141.84,2,...,-999.0,-999.0,-999.0,0.535985,0.680677,0.778348,-999.0,-999.000000,0.200000,1.000000


In [5]:
# train.columns

Index(['YearMonthDayHourMinute', 'STN', 'Lon', 'Lat', 'isitu-LST', 'SolarZA',
       'SateZA', 'ESR', 'Height', 'LandType', 'insitu-TA', 'Year', 'Month',
       'Day', 'Hour', 'Minute', 'Band1_new', 'Band2_new', 'Band3_new',
       'Band4_new', 'Band5_new', 'Band6_new', 'Band7_new', 'Band8_new',
       'Band9_new', 'Band10_new', 'Band11_new', 'Band12_new', 'Band13_new',
       'Band14_new', 'Band15_new', 'Band16_new', '30daysBand3_new',
       '30daysBand13_new', 'GK2A-LST_new', 'insitu-HM_new', 'insitu-TD_new',
       'insitu-TG_new', 'insitu-TED0.05_new', 'insitu-TED0.1_new',
       'insitu-TED0.2_new', 'insitu-TED0.3_new', 'insitu-TED0.5_new',
       'insitu-TED1.0_new', 'insitu-TED1.5_new', 'insitu-TED3.0_new',
       'insitu-TED5.0_new', 'insitu-PA_new', 'insitu-PS_new', 'mean_6_23_new',
       'mean_34_37_new', 'mean_38_39_new', 'mean_40_42_new', 'mean_43_44_new',
       'NDVI_new', 'NDWI_new'],
      dtype='object')

In [4]:
def model_train(curr_dir):
 
    path_train = os.path.join(curr_dir+"\\data_all\\"+"\\train")
    path_test = os.path.join(curr_dir+"\\data_all\\"+"\\test")
    
    folders_tr = os.listdir(path_train)
    folders_test = os.listdir(path_test)
    
    train = pd.DataFrame()
    for files in folders_tr:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\train\\")+files)
        train = pd.concat([train, df])            
              
    
    test = pd.DataFrame()
    for files in folders_test:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\test\\")+files)
        test = pd.concat([test, df])   
         

        
        
    X_train = train[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7',\
                     '30daysBand3','30daysBand13','GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM',\
                     'insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0',\
                     'mean_40_42','mean_43_44','NDVI', 'NDWI']]
    X_test = test[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7',\
                     '30daysBand3','30daysBand13','GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM',\
                     'insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0',\
                     'mean_40_42','mean_43_44','NDVI', 'NDWI']]
    
    y_train_LST=train[['isitu-LST']].values
    y_test_LST=test[['isitu-LST']].values
    
    y_train_TA=train[['insitu-TA']].values
    y_test_TA=test[['insitu-TA']].values
    
#     model_name = '\\save\\'+'best_model_v1.h5' 

#     filepath = os.path.join(os.getcwd()+ model_name)

#     es = EarlyStopping(monitor='val_mse', verbose=1, patience=20)

#     checkpoint = ModelCheckpoint(filepath=filepath,
#                              monitor='val_mse',
#                              verbose=1,
#                              save_best_only=True)

#     callbacks = [checkpoint, es]

    lgbm_reg_LST = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves)
    
    lgbm_reg_LST.fit(X_train, y_train_LST,
              eval_set=[(X_test, y_test_LST)],
              eval_metric=["mse"],
              early_stopping_rounds=early_stopping_rounds,
              verbose=verbose)
    
    lgbm_reg_TA = LGBMRegressor(objective= "regression",
                          max_depth= max_depth,
                          n_estimators= n_estimators,
                          learning_rate= lr,
                          num_leaves = num_leaves)
    
    lgbm_reg_TA.fit(X_train, y_train_TA,
              eval_set=[(X_test, y_test_TA)],
              eval_metric=["mse"],
              early_stopping_rounds=early_stopping_rounds,
              verbose=verbose) 
    
 
    
#     lgbm_reg.load_weights(model_name)
    
    
    path_val = os.path.join(curr_dir+"\\data_all\\"+"\\validation")
    folders_val = os.listdir(path_val)
    
    validation = pd.DataFrame()
    for files in folders_val:
        df= pd.read_csv(os.path.join(curr_dir+"\\data_all\\"+"\\validation\\")+files)
        validation = pd.concat([validation, df])        
        
    validation_ = validation[['Lat','SolarZA','SateZA','ESR','Band1','Band2','Band3','Band4','Band6','Band7',\
                     '30daysBand3','30daysBand13','GK2A-LST','insitu-TD','insitu-TG', 'insitu-HM',\
                     'insitu-PS','insitu-TED0.05','insitu-TED0.1','insitu-TED0.2',\
                     'insitu-TED0.3','insitu-TED0.5','insitu-TED1.0','insitu-TED1.5','insitu-TED5.0',\
                     'mean_40_42','mean_43_44','NDVI', 'NDWI']]
    prediction_LST = lgbm_reg_LST.predict(validation_)
    prediction_TA = lgbm_reg_TA.predict(validation_)
    
    validation_partial = validation[['YearMonthDayHourMinute', 'STN']]
    validation_partial = validation_partial.assign(LST = prediction_LST)
    validation_partial = validation_partial.assign(TA = prediction_TA)
    
    submission=pd.read_csv('1-2_검증데이터셋.csv', encoding='utf-8')
    submission1=submission.copy()
    submission1[['isitu-LST', 'insitu-TA']] = validation_partial[['LST','TA']].values       # 컬럼에 맞춰 넣음
    
    submission1.to_csv("lgbm_v2.csv", index = False)

In [5]:
model_train(curr_dir)

C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\younjeongOh\anaconda3\envs\tensorflow_env\lib\site-packages\lightgbm\basic.py:179: UserWarning: Converting column-vector to 1d array
  _log_warning('Converting column-vector to 1d array')


[500]	valid_0's l2: 4.59824
[1000]	valid_0's l2: 4.22111
[1500]	valid_0's l2: 3.98734
[2000]	valid_0's l2: 3.84708
[2500]	valid_0's l2: 3.73505
[3000]	valid_0's l2: 3.6613
[3500]	valid_0's l2: 3.60191
[4000]	valid_0's l2: 3.55878
[4500]	valid_0's l2: 3.51997
[5000]	valid_0's l2: 3.49147
[500]	valid_0's l2: 0.428702
[1000]	valid_0's l2: 0.249224
[1500]	valid_0's l2: 0.169492
[2000]	valid_0's l2: 0.125297
[2500]	valid_0's l2: 0.10029
[3000]	valid_0's l2: 0.0852353
[3500]	valid_0's l2: 0.0747801
[4000]	valid_0's l2: 0.0668963
[4500]	valid_0's l2: 0.061124
[5000]	valid_0's l2: 0.0564624
